<a href="https://colab.research.google.com/github/Rafal-Wawrzak/Fianal_One/blob/main/Project_1_Final_Linear_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#pandas
import pandas as pd

#Visual Exploration
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Meching Learning Items
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(display='diagram')

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import make_column_selector, make_column_transformer

from sklearn.tree import DecisionTreeRegressor

# Import the bagging regressor
from sklearn.ensemble import BaggingRegressor

# Import random forest Regressor
from sklearn.ensemble import RandomForestRegressor

In [26]:
#enter path for CSV, or Excel
path = '/content/drive/MyDrive/Boot  Camp/Project one/sales_predictions.csv'

In [27]:
##################################
# Comment out the one not needed #
##################################
df = pd.read_csv(path) #excel Load 

In [28]:
def evaluate_model(y_true, y_pred):
  """takes two arrays, true labels and predicted labels, and prints
  MAE, MSE, RMSE, and R2 metrics"""
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  r2 = r2_score(y_true, y_pred)
  rmse = np.sqrt(mse)
  print(f'MAE {mae}\n MSE {mse}\n R^2 {r2}\n RMSE {rmse}')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


#linear Regression

In [30]:
y = df['Item_Outlet_Sales']
X = df.drop(columns = 'Item_Outlet_Sales')
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [31]:
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

num_columns = num_selector(X_train)
cat_columns = cat_selector(X_train)

median_imputer = SimpleImputer(strategy='median')
freq_imputer = SimpleImputer(strategy='most_frequent')

median_imputer.fit(X_train[num_columns])
freq_imputer.fit(X_train[cat_columns])

X_train.loc[:, num_columns] = median_imputer.transform(X_train[num_columns])
X_test.loc[:, num_columns] = median_imputer.transform(X_test[num_columns])

X_train.loc[:, cat_columns] = freq_imputer.transform(X_train[cat_columns])
X_test.loc[:, cat_columns] = freq_imputer.transform(X_test[cat_columns])

scaler = StandardScaler()
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

num_tuple = (scaler, num_selector)
cat_tuple = (ohe_encoder, cat_selector)

col_trans = make_column_transformer(num_tuple, cat_tuple, remainder = 'passthrough')
lin_reg = LinearRegression()

pipe = make_pipeline(col_trans, lin_reg)

In [32]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fe002a29a50>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fe002a29a10>)])),
                ('linearregression', LinearRegression())])

In [33]:
evaluate_model(y_train, pipe.predict(X_train))

MAE 750.8202868585734
 MSE 998090.146792734
 R^2 0.6627453672423171
 RMSE 999.0446170180459
